# Recommender 

This recommender project uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms.

In [15]:
# %load /home/ubuntu/projects/initialize.py
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
app_name='app'
spark=SparkSession.builder.appName(app_name).getOrCreate()


Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

In [16]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [17]:
data = spark.read.csv('data/movielens_ratings.csv',inferSchema=True,header=True)

In [18]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [19]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [20]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [21]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see how the model performed.

In [22]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [23]:
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|     31|   4.0|    12|   0.2301091|
|     31|   1.0|    19|  0.90208876|
|     31|   3.0|     8|   2.1959877|
|     85|   1.0|    13|   3.5168304|
|     85|   5.0|    16|   3.4186137|
|     85|   3.0|    21|   2.3864803|
|     65|   1.0|    16|  -1.0209796|
|     53|   3.0|    20|   2.0802665|
|     78|   1.0|    27|  0.84140104|
|     78|   1.0|     2|   -0.833247|
|     34|   3.0|    25|   1.3445616|
|     34|   1.0|     0| -0.27896202|
|     81|   4.0|    11|  -1.9030857|
|     81|   3.0|    18|  0.77249664|
|     28|   1.0|    27|-0.017428888|
|     28|   1.0|     0|   1.5608784|
|     76|   1.0|     1|  -2.7133338|
|     76|   3.0|     3|   0.5973069|
|     26|   1.0|     6| 0.118736714|
|     26|   1.0|     3|-0.061883956|
+-------+------+------+------------+
only showing top 20 rows



In [24]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.8615765938717304


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would we actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [25]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [26]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     19|    11|
|     22|    11|
|     25|    11|
|     27|    11|
|     30|    11|
|     36|    11|
|     41|    11|
|     45|    11|
|     70|    11|
|     81|    11|
|     90|    11|
+-------+------+



In [27]:
reccomendations = model.transform(single_user)

In [28]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     19|    11|  4.6094747|
|     25|    11|  3.9128265|
|     36|    11|   2.470443|
|     45|    11|  1.9146473|
|     27|    11|  1.8388556|
|     22|    11|   1.420126|
|     90|    11| 0.83732325|
|     41|    11| 0.70224786|
|     70|    11|-0.22689545|
|     30|    11| -1.6252961|
|     81|    11| -1.9030857|
+-------+------+-----------+

